## Programación Lineal
### Ejercicio con Variables Indicadoras

En este ejercicio planteamos encontrar una cartera de acciones sujeto a una serie de restricciones que nos obligan a modelar el problema con variables indicadoras. Dada la rentabilidad esperada de cada acción y el peso que ocupa en el benchmark, queremos encontrar una cartera que maximice la rentabilidad esperada de la cartera sujeto a:
- Tener al menos 7 valores en cartera
- Todas las posiciones deben ser al menos del 3%
- Cada posición debe ser como máximo el doble del peso del índice


In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

### Datos
Tenemos disponibles un dataframe con los rendimientos esperados y los pesos 

In [2]:
df = pd.read_csv('../data/returns_and_weights.csv')

In [3]:
df

,tickers,expected_return,index_weight
0,SAN,0.04,0.12
1,BBVA,0.07,0.06
2,ITX,0.10,0.14
3,TEF,0.14,0.05
4,REP,0.07,0.03
5,IBE,0.10,0.11
6,CLNX,0.07,0.07
7,AMS,0.10,0.03
8,ELE,0.02,0.01
9,IAG,0.07,0.02


In [4]:
n = df.shape[0]
eret = df.expected_return.values
cap = df.index_weight.values

In [16]:
cap

array([0.12, 0.06, 0.14, 0.05, 0.03, 0.11, 0.07, 0.03, 0.01, 0.02, 0.04,
       0.05, 0.04, 0.03, 0.05, 0.08, 0.07, 0.04])

In [5]:
eret

array([0.04, 0.07, 0.1 , 0.14, 0.07, 0.1 , 0.07, 0.1 , 0.02, 0.07, 0.02,
       0.11, 0.09, 0.1 , 0.04, 0.03, 0.05, 0.03])

___


In [7]:
wg = cp.Variable(n)
objetivo = cp.Maximize(cp.sum(cp.multiply(eret, wg)))

In [17]:
constraints = [
    cp.sum(wg) == 1, 
    wg >= 0.03,
    wg <= 2*cap,
]

In [18]:
prob = cp.Problem(objetivo, constraints)
result = prob.solve()

In [19]:
result

-inf

In [15]:
wg.value

array([0.03      , 0.03      , 0.03      , 0.48999999, 0.03      ,
       0.03      , 0.03      , 0.03      , 0.03      , 0.03      ,
       0.03      , 0.03      , 0.03      , 0.03      , 0.03      ,
       0.03      , 0.03      , 0.03      ])

____

In [20]:
wg = cp.Variable(n)
d = cp.Variable(n, boolean=True)

In [21]:
objetivo = cp.Maximize(eret @ wg)

 $wg > 0 \implies d = 1$
 
 $d = 1 \implies wg >= 0.03$ 

 $wg + m d >= m + 0.03$
 
 $wg -0.03 d >= 0$  
 $wg >= 0.03 d$

In [22]:
constraints = [
    cp.sum(wg) <= 1,
    wg <= 2*cap,
    wg >= 0,
    cp.sum(d) >= 7,
    wg <= d,
    wg >= 0.03*d
]

In [23]:
prob = cp.Problem(objetivo, constraints)

In [24]:
res = prob.solve()

In [25]:
res

0.10120000000000001

In [26]:
pd.Series(wg.value, index=df.tickers).round(4)

tickers
SAN     0.00
BBVA    0.03
ITX     0.28
TEF     0.10
REP     0.03
IBE     0.22
CLNX    0.04
AMS     0.06
ELE    -0.00
IAG    -0.00
FER    -0.00
GRF     0.10
REE     0.08
ENG     0.06
NTGY   -0.00
ACS    -0.00
CABK   -0.00
ACX    -0.00
dtype: float64

In [27]:
d.value

array([0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0.])